In [1]:
from selenium import webdriver
import pandas as pd

In [2]:
driver = webdriver.Firefox()
url = 'https://www.seloger.com/list.htm?tri=initial&idtypebien=2,1,13,14,9&pxMax=500000&idtt=2,5&naturebien=1,2,4&ci=750109'
driver.get(url)

In [3]:
class appartement():
    
    def __init__(
        self,
        prix: int = None,
        superficie: float = None,
        nb_pieces: int = None,
        arrondissement: str = None,
        etage: int = None, 
        url: str = None
    ):
        self.prix = prix
        self.superficie = superficie        
        self.nb_pieces = nb_pieces
        self.arrondissement = arrondissement
        self.etage = etage
        self.url = url
        
    def to_dict(self):
        return {
            attr: getattr(self, attr) for attr in ['prix', 'superficie', 'nb_pieces', 'arrondissement', 'etage', 'url']
        }

In [4]:
def parse_criterion(text):
    
    crts = text.split(' ')
    nb_pieces, superficie, etage = None, None, None
    for i, x in enumerate(crts[::2]):
        if crts[2*i+1] == 'p':
            nb_pieces = int(x)
        elif crts[2*i+1] == 'm²':
            superficie = float(x.replace(',', '.'))
        elif crts[2*i+1] == 'etg':
            etage = int(x)
        
    return nb_pieces, superficie, etage

In [5]:
def load_appartment(element):
    price = element.find_element_by_class_name('c-pa-cprice')
    price = int(''.join(price.text.split(' ')).split('€')[0])
    
    arrondissement = element.find_element_by_class_name('c-pa-city').text
    
    url = element.find_element_by_class_name('c-pa-link').get_attribute('href')
    
    criterion = element.find_element_by_class_name('c-pa-criterion')
    nb_pieces, superficie, etage = parse_criterion(criterion.text)
    
    return appartement(
        prix = price,
        superficie = superficie,
        nb_pieces = nb_pieces,
        arrondissement = arrondissement,
        etage = etage,
        url = url
    )

In [22]:
def get_appartments(
    driver,
    base_url,
#     results: list = []
):

    driver.get(base_url)
    results = [load_appartment(x) for x in driver.find_elements_by_xpath('//*[@class="c-pa-list c-pa-sl cartouche "]')] 
    results += [load_appartment(x) for x in  driver.find_elements_by_xpath('//*[@class="c-pa-list c-pa-sl c-pa-gold cartouche "]')]
    
    next_page = driver.find_elements_by_xpath('//*[@class="pagination-next"]') 
    if next_page:
        results += get_appartments(
            driver,
            next_page[0].get_attribute('href'),
        )
    
    return results

In [23]:
appartements = get_appartments(driver, url)

In [24]:
results_df = pd.DataFrame(
    [x.to_dict() for x in appartements]
)

In [25]:
results_df['prix_m²'] = results_df['prix'] / results_df['superficie']

In [26]:
results_df.sort_values('prix_m²')

,arrondissement,etage,nb_pieces,prix,superficie,url,prix_m²
62,Paris 9ème,NaN,5,175700,88.37,https://www.seloger.com/annonces/viagers/appar...,1988.231300
45,Paris 9ème,NaN,4,250000,87.70,https://www.seloger.com/annonces/viagers/appar...,2850.627138
43,Paris 9ème,NaN,3,198960,65.00,https://www.seloger.com/annonces/viagers/appar...,3060.923077
15,Paris 9ème,NaN,6,475000,116.09,https://www.seloger.com/annonces/viagers/appar...,4091.653028
63,Paris 9ème,NaN,1,37000,7.60,https://www.seloger.com/annonces/achat/apparte...,4868.421053
73,Paris 9ème,NaN,1,39000,5.70,https://www.seloger.com/annonces/achat/apparte...,6842.105263
37,Paris 9ème,6.0,1,38000,5.42,https://www.seloger.com/annonces/achat/apparte...,7011.070111
39,Paris 9ème,NaN,1,80000,10.00,https://www.seloger.com/annonces/viagers/appar...,8000.000000
64,Paris 9ème,6.0,1,50000,6.00,https://www.seloger.com/annonces/achat/apparte...,8333.333333
3,Paris 9ème,4.0,1,151000,18.00,https://www.seloger.com/annonces/achat/apparte...,8388.888889


In [31]:
results_df.iloc[61]['url']

'https://www.seloger.com/annonces/achat/appartement/paris-9eme-75/145165807.htm?ci=750109&idtt=2,5&idtypebien=2,1,13,14,9&LISTING-LISTpg=4&naturebien=1,2,4&pxmax=500000&tri=initial&bd=ListToDetail'

In [ ]:
parse_criterion(elements[0].find_element_by_class_name('c-pa-criterion').text)

In [ ]:
for i, x in enumerate(['1', 'p', '26', 'm²'][::2]):
    print(x == 'm²')
    print(i)

In [ ]:
elements[0].text

In [ ]:
temp= elements[0].find_element_by_class_name('c-pa-link')

In [ ]:
temp.get_attribute('href')

In [ ]:
temp.text

In [ ]:
int(''.join(temp.text.split(' ')).split('€')[0])

In [ ]:
int(temp.text.split('p')[0]

In [ ]:
params = {
    'tri': 'initial',
    'dtypebien': [2,1,13,14,9],
    'pxMax' : 500000, 
    'idtt': [2,5], 
    'naturebien' : [1,2,4],
    'ci': [750109]
}